In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 
plt.style.use('dark_background')

import pandas as pd
import altair as alt 
import pathlib

from ipywidgets import HTML
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Marker, Popup

## data

In [2]:
lat = [47.641746116666674, 47.64200583333333, 47.645735516666676, 
       47.6451564, 47.64788463333333, 47.64902155,
       47.6519133535415, 47.65192425382816, 47.654193271383505, 
       47.6568559841464, 47.65645819801046, 47.6575144165449, 
       47.65827774385282, 47.65834086666667, 47.655170983333335,
       47.642047133333335, 47.644862033333325, 47.645739683333346,
       47.645741516666675, 47.645130249999994, 47.64512941666667,
       47.648307666666675, 47.649132316666666, 47.64894488333333, 
       47.65081421111258]

long = [-122.13970368333332, -122.14119843333334, -122.14181134999998, 
        -122.13819428333333, -122.13770626666668, -122.14218468333333,
        -122.14124879190584, -122.14120784481692, -122.14030990657797,
        -122.14092214876202, -122.1414173676043, -122.14149146668811,
        -122.14159648578442, -122.14156053333333, -122.14358155,
        -122.14114473333333, -122.14247085000001, -122.14187858333334,
        -122.14187855, -122.13816931666668, -122.138172,
        -122.13744040000002, -122.13748775, -122.13738295,
        -122.14015336943993]

rssi = [-87, -115, -129, -129, -128, -126, -140, -137, -140,-145, -145,
        -141, -145, -142, -147, -101, -130, -126, -128, -113, -116,
        -128, -131, -136, -139]

SF = [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
      12, 12, 7, 9, 9, 8, 8, 7, 12, 11, 12, 12]

BW = [9, 9, 9, 9, 9, 9, 7, 8, 8, 7, 7, 6, 6,
      3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5]

name = [f'location-{i}' for i in range(len(lat))]


# plotting adjustment 
x_adj = [
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0,
]

y_adj = [
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 
    0, 0.015, 0, 0, 0, 
    0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0,
]

df_data = {
    'lat': lat,
    'lon': long,
    'rssi': rssi,
    'SF': SF,
    'BW': BW,
    'name': name,
#     'x_adj': x_adj,
#     'y_adj': y_adj,
}

rssi_df = (
    pd.DataFrame(df_data)
#     .assign(
#         lat_plot = lambda df: df.lat + df.y_adj,
#         lon_plot = lambda df: df.lon + df.x_adj,
#     )
)
rssi_df.head()

,lat,lon,rssi,SF,BW,name
0,47.641746,-122.139704,-87,12,9,location-0
1,47.642006,-122.141198,-115,12,9,location-1
2,47.645736,-122.141811,-129,12,9,location-2
3,47.645156,-122.138194,-129,12,9,location-3
4,47.647885,-122.137706,-128,12,9,location-4


In [3]:
rssi_df.sort_values('lat')

,lat,lon,rssi,SF,BW,name
0,47.641746,-122.139704,-87,12,9,location-0
1,47.642006,-122.141198,-115,12,9,location-1
15,47.642047,-122.141145,-101,7,9,location-15
16,47.644862,-122.142471,-130,9,9,location-16
20,47.645129,-122.138172,-116,7,9,location-20
19,47.645130,-122.138169,-113,8,9,location-19
3,47.645156,-122.138194,-129,12,9,location-3
2,47.645736,-122.141811,-129,12,9,location-2
17,47.645740,-122.141879,-126,9,9,location-17
18,47.645742,-122.141879,-128,8,9,location-18


## pydeck

In [4]:
import pydeck as pdk 

mid_lat, mid_lon = rssi_df.query('name == "location-5"')[['lat', 'lon']].values.tolist()[0]
mid_lat, mid_lon

(47.64902155, -122.14218468333333)

In [5]:
r = pdk.Deck(
    # map_style="mapbox://styles/mapbox/light-v9",
    map_style="mapbox://styles/zkapetanovic/ck83iz9cq0lpr1ip7dbcj0kvv",
    mapbox_key="pk.eyJ1IjoiemthcGV0YW5vdmljIiwiYSI6ImNrODNhMXU5czBtcm4zZW10aXdnYmNrM3QifQ.N7LKtouGF0zRcEMXdCmUpA",
    initial_view_state={
        "latitude": mid_lat,
        "longitude": mid_lon,
        "zoom": 11,
        "pitch": 50,
    },
    layers=[
        pdk.Layer(
            "HexagonLayer",
            data=rssi_df,
            get_position=["lon", "lat"],
            radius=100,
            elevation_scale=4,
            elevation_range=[0, 1000],
            pickable=True,
            extruded=True,
        ),
    ],
)

r.show()

DeckGLWidget(json_input='{"initialViewState": {"latitude": 47.64902155, "longitude": -122.14218468333333, "pit…

## altair

### setup 

In [ ]:
# topojson_url = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
# wa_topo_source = alt.topo_feature(topojson_url, "cb_2015_washington_county_20m")

# topojson_url = 'https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/city-council-districts.topojson'
# wa_topo_source = alt.topo_feature(topojson_url, "geo_export_bb6ddf4f-ad18-4c66-9789-b4dc933df7cc")

topojson_url = 'https://opendata.arcgis.com/datasets/7ec97be7bec2443e92ad948b3d967a26_0.geojson'
wa_topo_source = alt.topo_feature(topojson_url, "geo_export_bb6ddf4f-ad18-4c66-9789-b4dc933df7cc")

### map

In [ ]:
lat = [47.641746116666674, 47.64200583333333, 47.645735516666676, 
       47.6451564, 47.64788463333333, 47.64902155,
       47.6519133535415, 47.65192425382816, 47.654193271383505, 
       47.6568559841464, 47.65645819801046, 47.6575144165449, 
       47.65827774385282, 47.65834086666667, 47.655170983333335,
       47.642047133333335, 47.644862033333325, 47.645739683333346,
       47.645741516666675, 47.645130249999994, 47.64512941666667,
       47.648307666666675, 47.649132316666666, 47.64894488333333, 
       47.65081421111258]

long = [-122.13970368333332, -122.14119843333334, -122.14181134999998, 
        -122.13819428333333, -122.13770626666668, -122.14218468333333,
        -122.14124879190584, -122.14120784481692, -122.14030990657797,
        -122.14092214876202, -122.1414173676043, -122.14149146668811,
        -122.14159648578442, -122.14156053333333, -122.14358155,
        -122.14114473333333, -122.14247085000001, -122.14187858333334,
        -122.14187855, -122.13816931666668, -122.138172,
        -122.13744040000002, -122.13748775, -122.13738295,
        -122.14015336943993]

rssi = [-87, -115, -129, -129, -128, -126, -140, -137, -140,-145, -145,
        -141, -145, -142, -147, -101, -130, -126, -128, -113, -116,
        -128, -131, -136, -139]

SF = [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
      12, 12, 7, 9, 9, 8, 8, 7, 12, 11, 12, 12]

BW = [9, 9, 9, 9, 9, 9, 7, 8, 8, 7, 7, 6, 6,
      3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5]

name = [f'location-{i}' for i in range(len(lat))]


# plotting adjustment 
x_adj = [
    0, -9, 10, 11, -8, 
    0, -8, 8, 0, 9, 
    -8, 0, -10, 10, 0, 
    9, 0, 11, -10, 11, 
    -10, 10, 10, -10, 0,
]

y_adj = [
    0, -4, -2, -3, -5, 
    0, -5, 2, 0, 2, 
    -1, 0, -5, 2, 0, 
    2, 0, 0, -2, 3, 
    -2, 1, 1, -4, 0,
]

df_data = {
    'lat': lat,
    'lon': long,
    'rssi': rssi,
    'SF': SF,
    'BW': BW,
    'name': name,
    'x_adj': x_adj,
    'y_adj': y_adj,
}

rssi_df = (
    pd.DataFrame(df_data)
    .assign(
        lat_plot = lambda df: df.lat + df.y_adj*0.0001,
        lon_plot = lambda df: df.lon + df.x_adj*0.0001,
    )
)
rssi_df.head()

In [ ]:
chart_df = rssi_df

circles = alt.Chart(
    chart_df
).mark_circle(
    opacity=0.6,
    size=50
).encode(
    latitude='lat:Q',
    longitude='lon:Q',
#     size='rssi:Q',
    color=alt.Color('name:N', scale=alt.Scale(scheme='tableau20')),
    tooltip='name:N',
)

# display(circles)

labels = alt.Chart(
    chart_df
).mark_text(
    align='center',
    baseline='bottom',
    dx=0, dy=-5 ,
    fontSize=10,
).encode(
    latitude='lat_plot:Q',
    longitude='lon_plot:Q',
    text='name:N',
)

# display(labels)

layered = alt.layer(
#     county_map, 
    circles, labels,
).properties(
    width=500, height=700,
).configure_view(
    stroke=None
).project(type='albers')


display(layered)

In [ ]:
layered.save('charts/test-chart.html')

In [ ]:
layered.save('charts/test-chart.png')

In [ ]:
alt.layer(
    alt.Chart(wa_topo_source).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ).transform_filter('datum.properties.NAME == "King"'),
    alt.Chart(rssi_df).mark_circle(size=9).encode(
        latitude='lat:Q',
        longitude='lon:Q',
        tooltip='rssi:N'
    )
# ).project(
#     type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

## map dev 

In [ ]:
def _generate_popup(lat, lon, message='default'):
    text = HTML(value=message)
    popup = Popup(
        location=(lat, lon), child=text,
        max_height=15, max_width=15,
        close_button=False, auto_close=False, close_on_escape_key=False
    )
    
    return popup
    

def _generate_marker(lat, lon):
    marker = Marker(location=(lat, lon))    
    return marker


def _generate_circle_marker(lat, lon):
    marker = CircleMarker(location=(lat, lon), radius=15, color='red')    
    return marker
    

m = Map(basemap=basemaps.Hydda.Base, center=(47.650682, -122.140007), zoom=15)

N = len(long)
for i in range(N):
#     marker = _generate_marker(lat[i], long[i])
    marker = _generate_circle_marker(lat[i], long[i])
    popup = _generate_popup(lat[i], long[i], message='default')

    m.add_layer(marker)
    m.add_layer(popup)


m